In [1]:
from google.colab import files 
uploaded = files.upload() 

Saving data.csv to data.csv


In [2]:
import io 
import pandas as pd
data = pd.read_csv(io.BytesIO(uploaded['data.csv'])) 
print(type(data))
print(data.head())

<class 'pandas.core.frame.DataFrame'>
                                                URLs  ... Label
0  http://www.bbc.com/news/world-us-canada-414191...  ...     1
1  https://www.reuters.com/article/us-filmfestiva...  ...     1
2  https://www.nytimes.com/2017/10/09/us/politics...  ...     1
3  https://www.reuters.com/article/us-mexico-oil-...  ...     1
4  http://www.cnn.com/videos/cnnmoney/2017/10/08/...  ...     1

[5 rows x 4 columns]


In [3]:
url_data=data.dropna()
url_data.head()

,URLs,Headline,Body,Label
0,http://www.bbc.com/news/world-us-canada-414191...,Four ways Bob Corker skewered Donald Trump,Image copyright Getty Images\nOn Sunday mornin...,1
1,https://www.reuters.com/article/us-filmfestiva...,Linklater's war veteran comedy speaks to moder...,"LONDON (Reuters) - “Last Flag Flying”, a comed...",1
2,https://www.nytimes.com/2017/10/09/us/politics...,Trump’s Fight With Corker Jeopardizes His Legi...,The feud broke into public view last week when...,1
3,https://www.reuters.com/article/us-mexico-oil-...,Egypt's Cheiron wins tie-up with Pemex for Mex...,MEXICO CITY (Reuters) - Egypt’s Cheiron Holdin...,1
4,http://www.cnn.com/videos/cnnmoney/2017/10/08/...,Jason Aldean opens 'SNL' with Vegas tribute,"Country singer Jason Aldean, who was performin...",1


In [4]:
url_data.shape

(3988, 4)

In [5]:
data_x=url_data.iloc[:,:-1]
data_x.head()

,URLs,Headline,Body
0,http://www.bbc.com/news/world-us-canada-414191...,Four ways Bob Corker skewered Donald Trump,Image copyright Getty Images\nOn Sunday mornin...
1,https://www.reuters.com/article/us-filmfestiva...,Linklater's war veteran comedy speaks to moder...,"LONDON (Reuters) - “Last Flag Flying”, a comed..."
2,https://www.nytimes.com/2017/10/09/us/politics...,Trump’s Fight With Corker Jeopardizes His Legi...,The feud broke into public view last week when...
3,https://www.reuters.com/article/us-mexico-oil-...,Egypt's Cheiron wins tie-up with Pemex for Mex...,MEXICO CITY (Reuters) - Egypt’s Cheiron Holdin...
4,http://www.cnn.com/videos/cnnmoney/2017/10/08/...,Jason Aldean opens 'SNL' with Vegas tribute,"Country singer Jason Aldean, who was performin..."


In [6]:
data_y=url_data.iloc[:,3:]
data_y.head()

,Label
0,1
1,1
2,1
3,1
4,1


In [7]:
print(data_x.shape)
print(data_y.shape)

(3988, 3)
(3988, 1)


In [0]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
#from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from matplotlib import pyplot as plt

In [0]:
def split_data(x,y):
    
    """
    Reminder : Clean the set before sending here.
    Input  : x=Attributes of the dataset, y=Target Attribute
    Output : Split data of train test
    
    We are removing dev , as dev can be better used for the models that includes DNN, CNN and RNN
    
    """
    X=x
    Y=y
    
    train_x,test_x,train_y,test_y = train_test_split(x,y,test_size=0.2,random_state=12)
    
    #train_x : Training data with attributes
    #train_y : Training data with ratings
    #test_x  : testing data with attributes
    #test_y  : testing data with rating 
    
    print("Final size of train/test :",train_x.size,"/",test_x.size)
    return train_x,train_y,test_x,test_y

In [0]:
#navie_bayes
from sklearn.model_selection import GridSearchCV
def navie_bayes(train_x,train_y,test_x,test_y,itera=100):  #itera changed to 100
   
    print('model building')
    NBG = MultinomialNB()
    if itera == 100:
        alphas=np.arange(0.001,1,0.001)
    else:
        alphas=np.arange(0.01,1,0.01)
    print('begin grid')
    grid = GridSearchCV(estimator=NBG, param_grid=dict(alpha=alphas))

    grid.fit(train_x, train_y)
    
    #print(grid)
    # summarize the results of the grid search
    print("grid score",grid.best_score_)
    print("grid alpha",grid.best_estimator_.alpha)
    alpha=grid.best_estimator_.alpha
    
    NB = MultinomialNB(alpha = alpha )
    NB.fit(train_x,train_y)  
    predict_NB_train = NB.predict(train_x) 
    predict_NB_test = NB.predict(test_x) 
    score_train = metrics.accuracy_score(train_y, predict_NB_train) 
    score_test = metrics.accuracy_score(test_y, predict_NB_test)
      
    print("Alpha: {:.3f} test_Score: {:.5f} train_score: {:.5f}".format(alpha,score_test,score_train))# checking tha accuracy 
       
    #print("best accuracy with Alpha: {:.2f} Score: {:.5f}".format(alpha_value, best_score))
    cm_train= metrics.confusion_matrix(train_y, predict_NB_train)
    print("training confusion matrix")
    print(cm_train)
    cm_test = metrics.confusion_matrix(test_y, predict_NB_test)
    print("testing confusion matrix")
    print(cm_test)
    
    print("------------------------------------------")


In [11]:
import numpy as np
a=np.array([])
a.size

0

In [0]:
#mult_tfidf start 
def mult_tfidf(x1,y):
    from sklearn.feature_extraction.text import TfidfVectorizer

    #if x3.size==0:
    train_x1,train_y1,test_x1,test_y1=split_data(x1,y)
        #train_x2,train_y2,test_x2,test_y2=split_data(x2,y)



    tfidf_vectorizer = TfidfVectorizer(stop_words='english',max_df=0.7,min_df=0.02)   # a TFIDF vectorizer
    tfidf_train_x1 = pd.DataFrame(tfidf_vectorizer.fit_transform(train_x1).todense())  #fitting the training data
    tfidf_test_x1 = pd.DataFrame(tfidf_vectorizer.transform(test_x1).todense())

        #tfidf_train_x2 = pd.DataFrame(tfidf_vectorizer.fit_transform(train_x2).todense())  #fitting the training data
        #tfidf_test_x2 = pd.DataFrame(tfidf_vectorizer.transform(test_x2).todense()) 

        #train=pd.concat([tfidf_train_x1,tfidf_train_x2],axis=1)
        #test=pd.concat([tfidf_test_x1,tfidf_test_x2],axis=1)
    print(tfidf_train_x1.shape)
    print(tfidf_test_x1.shape)
       # print(train_y2.shape)
        #print(test_y2.shape)

    navie_bayes(tfidf_train_x1,train_y1,tfidf_test_x1,test_y1)
    
    '''else:
        train_x1,train_y1,test_x1,test_y1=an.split_data(x1,y)
        train_x2,train_y2,test_x2,test_y2=an.split_data(x2,y)
        train_x3,train_y3,test_x3,test_y3=an.split_data(x3,y)
        
        tfidf_vectorizer = TfidfVectorizer(stop_words='english',max_df=0.7,min_df=0.02)   # a TFIDF vectorizer
        tfidf_train_x1 = pd.DataFrame(tfidf_vectorizer.fit_transform(train_x1).todense())  #fitting the training data
        tfidf_test_x1 = pd.DataFrame(tfidf_vectorizer.transform(test_x1).todense())

        tfidf_train_x2 = pd.DataFrame(tfidf_vectorizer.fit_transform(train_x2).todense())  #fitting the training data
        tfidf_test_x2 = pd.DataFrame(tfidf_vectorizer.transform(test_x2).todense()) 
        
        tfidf_train_x3 = pd.DataFrame(tfidf_vectorizer.fit_transform(train_x3).todense())  #fitting the training data
        tfidf_test_x3 = pd.DataFrame(tfidf_vectorizer.transform(test_x3).todense())
        
        train=pd.concat([tfidf_train_x1,tfidf_train_x2,tfidf_train_x3],axis=1)
        test=pd.concat([tfidf_test_x1,tfidf_test_x2,tfidf_test_x3],axis=1)
        
        navie_bayes(train,train_y2,test,test_y2)'''

In [14]:
mult_tfidf(x1=data_x.URLs,y=data_y.Label)  #single attr URL

Final size of train/test : 3190 / 798
(3190, 46)
(798, 46)
model building
begin grid
grid score 0.9852664576802507
grid alpha 0.001
Alpha: 0.001 test_Score: 0.98371 train_score: 0.98527
training confusion matrix
[[1687    1]
 [  46 1456]]
testing confusion matrix
[[431   1]
 [ 12 354]]
------------------------------------------


In [15]:
mult_tfidf(x1=data_x.Headline,y=data_y.Label) #single attr headline

Final size of train/test : 3190 / 798
(3190, 14)
(798, 14)
model building
begin grid
grid score 0.5501567398119123
grid alpha 0.001
Alpha: 0.001 test_Score: 0.55388 train_score: 0.55298
training confusion matrix
[[1431  257]
 [1169  333]]
testing confusion matrix
[[367  65]
 [291  75]]
------------------------------------------


In [16]:
mult_tfidf(x1=data_x.Body,y=data_y.Label)

Final size of train/test : 3190 / 798
(3190, 1945)
(798, 1945)
model building
begin grid
grid score 0.925078369905956
grid alpha 0.936
Alpha: 0.936 test_Score: 0.88972 train_score: 0.93260
training confusion matrix
[[1509  179]
 [  36 1466]]
testing confusion matrix
[[361  71]
 [ 17 349]]
------------------------------------------


In [0]:
mult_tfidf(x1=data_x.URLs,x2=data_x.Headline,y=data_y.Label,x3=data_x.Body)

"""
Final size of train/test : 3190 / 798
Final size of train/test : 3190 / 798
Final size of train/test : 3190 / 798
model building
begin grid
grid score 0.9771159874608151
grid alpha 0.668
Alpha: 0.668 test_Score: 0.96867 train_score: 0.99530
training confusion matrix
[[1680    8]
 [   7 1495]]
testing confusion matrix
[[411  21]
 [  4 362]]
--------------------

"""

In [0]:
mult_tfidf(x1=data_x.URLs,x2=data_x.Headline,y=data_y.Label,x3=a)

Final size of train/test : 3190 / 798
Final size of train/test : 3190 / 798
(3190, 60)
(798, 60)
(3190,)
(798,)
model building
begin grid
grid score 0.9846394984326018
grid alpha 0.001
Alpha: 0.001 test_Score: 0.98246 train_score: 0.98527
training confusion matrix
[[1682    6]
 [  41 1461]]
testing confusion matrix
[[429   3]
 [ 11 355]]
------------------------------------------


In [0]:
mult_tfidf(x1=data_x.URLs,x2=data_x.Body,y=data_y.Label,x3=a)

Final size of train/test : 3190 / 798
Final size of train/test : 3190 / 798
(3190, 1991)
(798, 1991)
(3190,)
(798,)
model building
begin grid
grid score 0.9896551724137931
grid alpha 0.001
Alpha: 0.001 test_Score: 0.98496 train_score: 0.99216
training confusion matrix
[[1667   21]
 [   4 1498]]
testing confusion matrix
[[421  11]
 [  1 365]]
------------------------------------------


In [0]:
mult_tfidf(x1=data_x.Headline,x2=data_x.Body,y=data_y.Label,x3=a)

Final size of train/test : 3190 / 798
Final size of train/test : 3190 / 798
(3190, 1959)
(798, 1959)
(3190,)
(798,)
model building
begin grid
grid score 0.9109717868338558
grid alpha 0.001
Alpha: 0.001 test_Score: 0.88471 train_score: 0.92069
training confusion matrix
[[1471  217]
 [  36 1466]]
testing confusion matrix
[[354  78]
 [ 14 352]]
------------------------------------------


In [0]:
#multi_hashv start 
def multi_hashv(x1,x2,y,x3=a):
    from sklearn.feature_extraction.text import HashingVectorizer
    # Not sure about this, collisions may occur as I forced absolute cause naivie bayes is a bitch and n_features over 
                                                                                        #2**15 causes memory error
    
    if x3.size==0:
        train_x1,train_y1,test_x1,test_y1=split_data(x1,y)
        train_x2,train_y2,test_x2,test_y2=split_data(x2,y)



        hash_vectorizer = HashingVectorizer(stop_words='english',n_features = 2**8)   # a HASH vectorizer
        hash_train_x1 = pd.DataFrame(np.absolute(hash_vectorizer.fit_transform(train_x1).todense()))  #fitting the training data
        hash_test_x1 = pd.DataFrame(np.absolute(hash_vectorizer.transform(test_x1).todense()))

        hash_train_x2 = pd.DataFrame(np.absolute(hash_vectorizer.fit_transform(train_x2).todense()))  #fitting the training data
        hash_test_x2 = pd.DataFrame(np.absolute(hash_vectorizer.transform(test_x2).todense())) 

        train=pd.concat([hash_train_x1,hash_train_x2],axis=1)
        test=pd.concat([hash_test_x1,hash_test_x2],axis=1)
        print(train.shape)
        print(test.shape)
        print(train_y2.shape)
        print(test_y2.shape)

        navie_bayes(train,train_y2,test,test_y2,100)
    
    else:
        train_x1,train_y1,test_x1,test_y1=split_data(x1,y)
        train_x2,train_y2,test_x2,test_y2=split_data(x2,y)
        train_x3,train_y3,test_x3,test_y3=split_data(x3,y)
        
        hash_vectorizer = HashingVectorizer(stop_words='english',n_features = 2**8)   # a HASH vectorizer
        hash_train_x1 = pd.DataFrame(np.absolute(hash_vectorizer.fit_transform(train_x1).todense()) ) #fitting the training data
        hash_test_x1 = pd.DataFrame(np.absolute(hash_vectorizer.transform(test_x1).todense()))

        hash_train_x2 = pd.DataFrame(np.absolute(hash_vectorizer.fit_transform(train_x2).todense()) ) #fitting the training data
        hash_test_x2 = pd.DataFrame(np.absolute(hash_vectorizer.transform(test_x2).todense()) )
        
        hash_train_x3 = pd.DataFrame(np.absolute(hash_vectorizer.fit_transform(train_x3).todense()))  #fitting the training data
        hash_test_x3 = pd.DataFrame(np.absolute(hash_vectorizer.transform(test_x3).todense()))
        
        train=pd.concat([hash_train_x1,hash_train_x2,hash_train_x3],axis=1)
        test=pd.concat([hash_test_x1,hash_test_x2,hash_test_x3],axis=1)
        
        navie_bayes(train,train_y2,test,test_y2,100)


In [0]:
multi_hashv(x1=data_x.URLs,x2=data_x.Headline,y=data_y.Label,x3=data_x.Body)
"""
Final size of train/test : 3190 / 798
Final size of train/test : 3190 / 798
Final size of train/test : 3190 / 798
model building
begin grid
grid score 0.9432601880877742
grid alpha 0.553
Alpha: 0.553 test_Score: 0.92481 train_score: 0.94796
training confusion matrix
[[1535  153]
 [  13 1489]]
testing confusion matrix
[[376  56]
 [  4 362]]

"""

In [0]:
multi_hashv(x1=data_x.URLs,x2=data_x.Headline,y=data_y.Label,x3=a)

Final size of train/test : 3190 / 798
Final size of train/test : 3190 / 798
(3190, 512)
(798, 512)
(3190,)
(798,)
model building
begin grid
grid score 0.9510971786833856
grid alpha 0.794
Alpha: 0.794 test_Score: 0.94236 train_score: 0.96082
training confusion matrix
[[1599   89]
 [  36 1466]]
testing confusion matrix
[[396  36]
 [ 10 356]]
------------------------------------------


In [0]:
multi_hashv(x1=data_x.URLs,x2=data_x.Body,y=data_y.Label,x3=a)

Final size of train/test : 3190 / 798
Final size of train/test : 3190 / 798
(3190, 512)
(798, 512)
(3190,)
(798,)
model building
begin grid
grid score 0.95423197492163
grid alpha 0.001
Alpha: 0.001 test_Score: 0.94486 train_score: 0.95611
training confusion matrix
[[1559  129]
 [  11 1491]]
testing confusion matrix
[[390  42]
 [  2 364]]
------------------------------------------


In [0]:
multi_hashv(x1=data_x.Headline,x2=data_x.Body,y=data_y.Label,x3=a)

Final size of train/test : 3190 / 798
Final size of train/test : 3190 / 798
(3190, 512)
(798, 512)
(3190,)
(798,)
model building
begin grid
grid score 0.8376175548589341
grid alpha 0.443
Alpha: 0.443 test_Score: 0.80702 train_score: 0.85893
training confusion matrix
[[1352  336]
 [ 114 1388]]
testing confusion matrix
[[321 111]
 [ 43 323]]
------------------------------------------
